# `Промышленное машинное обучение на Spark`
## `Задание 02: Spark DataFrame`

<span style="color:red">Дедлайн: . 00:00</span>

<span style="color:red">Заполненный ноутбук присылать на почту <b>ekolmagorov98@yandex.ru</b> с темой письма <b>[HSE Spark 2024][Задание 02][ФИО]</b>.</span>

### `Часть 1`

In [1]:
! pip3 install pyspark pyarrow wget

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.3/38.3 MB 23.6 MB/s eta 0:00:0000:0100:01
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.2/18.2 MB 30.3 MB/s eta 0:00:0000:0100:01
  Created wheel for wget: filename=wget-3.2-py3-none-any.whl size=9675 sha256=4d765d25d8b1579b7a0e95a4907d0c3f7edbd860b4dfe2d9cfb15500a48c69c6
  Stored in directory: /home/diwert/.cache/pip/wheels/8b/f1/7f/5c94f0a7a505ca1c81cd1d9208ae2064675d97582078e6c769
Successfully built wget


Скачаем и распакуем данные:

In [3]:
import wget
import zipfile

filename = wget.download(
    'https://archive.ics.uci.edu/ml/machine-learning-databases/00228/smsspamcollection.zip'
)
with zipfile.ZipFile(filename, 'r') as zip_ref:
    zip_ref.extractall('./smsspamcollection')

-1 / unknown

In [2]:
import pyspark.sql.types as T
import pyspark.sql.functions as F
from pyspark.sql import SparkSession

from pyspark import SparkConf, SparkContext

conf = (
    SparkConf()
        .set('spark.ui.port', '4050')
        .setMaster('local[*]')
)
sc = SparkContext(conf=conf)
spark = SparkSession(sc)

24/02/11 20:47:51 WARN Utils: Your hostname, vm-03 resolves to a loopback address: 127.0.1.1; using 10.128.0.8 instead (on interface eth0)
24/02/11 20:47:51 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/02/11 20:47:52 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


**Задание 01:** Считайте DataFrame:

In [3]:
### YOUR CODE HERE

path = './smsspamcollection/SMSSpamCollection'
spam_df = spark.read.csv(path, sep='\t', inferSchema=True, header=False)
spam_df

DataFrame[_c0: string, _c1: string]

**Задание 02:** Назовите колонки таблицы `class` и `text`:

In [4]:
### YOUR CODE HERE
spam_df = spam_df.withColumnRenamed("_c0", "class").withColumnRenamed("_c1", "text")
spam_df

DataFrame[class: string, text: string]

**Задание 03:** Выведите первые $3$ строки датасета:

In [5]:
### YOUR CODE HERE
spam_df.show(3)

+-----+--------------------+
|class|                text|
+-----+--------------------+
|  ham|Go until jurong p...|
|  ham|Ok lar... Joking ...|
| spam|Free entry in 2 a...|
+-----+--------------------+
only showing top 3 rows



**Задание 04:** Создайте две новых колонки: `words` — список слов в тексте и `length` — длина данного списка. Реализуйте запрос за одно присваивание:

In [6]:
### YOUR CODE HERE
spam_df = (
            spam_df
                .withColumn('words',  F.split(spam_df['text'], ' '))
                .withColumn('length', F.size('words'))
           )
spam_df.show(3)

+-----+--------------------+--------------------+------+
|class|                text|               words|length|
+-----+--------------------+--------------------+------+
|  ham|Go until jurong p...|[Go, until, juron...|    20|
|  ham|Ok lar... Joking ...|[Ok, lar..., Joki...|     6|
| spam|Free entry in 2 a...|[Free, entry, in,...|    28|
+-----+--------------------+--------------------+------+
only showing top 3 rows



**Задание 05:** Посчитайте среднюю длину текстов в зависимости от класса:

In [7]:
### YOUR CODE HERE
spam_df.groupBy('class').agg(F.avg('length').alias('mean_length')).show()

+-----+------------------+
|class|       mean_length|
+-----+------------------+
|  ham|14.427594779366066|
| spam| 23.95314591700134|
+-----+------------------+



**Задание 06:** Выведите примерно $0.1\%$ самых длинных текстов:

In [8]:
### YOUR CODE HERE
spam_df.sort('length', ascending=False).show(int(spam_df.count() * 0.001))

+-----+--------------------+--------------------+------+
|class|                text|               words|length|
+-----+--------------------+--------------------+------+
|  ham|For me the love s...|[For, me, the, lo...|   171|
|  ham|The last thing i ...|[The, last, thing...|   162|
|  ham|Sad story of a Ma...|[Sad, story, of, ...|   125|
|  ham|Sad story of a Ma...|[Sad, story, of, ...|   125|
|  ham|How to Make a gir...|[How, to, Make, a...|   121|
+-----+--------------------+--------------------+------+
only showing top 5 rows



**Задание 07:** Найдите $10$ слов с самой высокой Document Frequency (количество смс, в которых слово встречается хотя бы раз):

In [9]:
### YOUR CODE HERE
counts_df = ( 
                spam_df
                    .select(F.explode(F.col("words")).alias("word"), F.col('text'))
                    .groupby("word")
                    .agg(F.countDistinct("text").alias("count"))
                    .sort(F.col("count").desc())
           )
counts_df.show(10)

+----+-----+
|word|count|
+----+-----+
|  to| 1514|
| you| 1168|
|   I| 1060|
|   a| 1000|
| the|  897|
| and|  667|
|  in|  666|
|  is|  597|
|   i|  573|
| for|  539|
+----+-----+
only showing top 10 rows



**Задание 08:** Положим, что заданы две матрицы $A$, $B$ в координатном виде. Реализуйте перемножение матриц:

In [10]:
# 4 x 3
A_data = [
    (0, 0, 8), (0, 1, 5), (0, 2, 8),
    (1, 0, 2), (1, 1, 5), (1, 2, 6),
    (2, 0, 2), (2, 1, 6), (2, 2, 4),
    (3, 0, 6), (3, 1, 3), (3, 2, 3),
]

# 3 x 5
B_data = [
    (0, 0, 9), (0, 1, 4), (0, 2, 9), (0, 3, 9), (0, 4, 1),
    (1, 0, 2), (1, 1, 5), (1, 2, 7), (1, 3, 3), (1, 4, 0),
    (2, 0, 9), (2, 1, 5), (2, 2, 7), (2, 3, 9), (2, 4, 0),
]

A = spark.createDataFrame(A_data, schema=['i', 'j', 'v'])
B = spark.createDataFrame(B_data, schema=['i', 'j', 'v'])

### YOUR CODE HERE
# Преобразуем матрицу B для удобства перемножения
B_transformed = B.selectExpr('i as i_B', 'j as j_B', 'v as v_B')

# Создаем все возможные комбинации строк из A и B_transformed
cross_joined = A.join(B_transformed)

# Отфильтровываем строки, где j (из A) не равно i_B (из B_transformed)
filtered = cross_joined.filter(A.j == B_transformed.i_B)

# Вычисляем произведение элементов и суммируем результаты
C = (
        filtered
            .withColumn('v', filtered['v'] * filtered['v_B'])
            .groupBy(A.i, B_transformed.j_B)
            .agg(F.sum('v').alias('v'))
            .withColumnRenamed('j_B', 'j')
            .sort(F.col('i'), F.col('j'))
    )


C.show()

+---+---+---+
|  i|  j|  v|
+---+---+---+
|  0|  0|154|
|  0|  1| 97|
|  0|  2|163|
|  0|  3|159|
|  0|  4|  8|
|  1|  0| 82|
|  1|  1| 63|
|  1|  2| 95|
|  1|  3| 87|
|  1|  4|  2|
|  2|  0| 66|
|  2|  1| 58|
|  2|  2| 88|
|  2|  3| 72|
|  2|  4|  2|
|  3|  0| 87|
|  3|  1| 54|
|  3|  2| 96|
|  3|  3| 90|
|  3|  4|  6|
+---+---+---+



**Задание 09**: Реализуйте перемножение тех же матриц, что и в 8 задании, но используя `spark.sql`:

In [11]:
### YOUR CODE HERE
A.createOrReplaceTempView("A")
B.createOrReplaceTempView("B")

C = spark.sql("""
    SELECT A.i AS i, B.j AS j, SUM(A.v * B.v) AS v
    FROM A
    JOIN B ON A.j = B.i
    GROUP BY A.i, B.j
""")
C.sort(F.col('i'), F.col('j')).show()

+---+---+---+
|  i|  j|  v|
+---+---+---+
|  0|  0|154|
|  0|  1| 97|
|  0|  2|163|
|  0|  3|159|
|  0|  4|  8|
|  1|  0| 82|
|  1|  1| 63|
|  1|  2| 95|
|  1|  3| 87|
|  1|  4|  2|
|  2|  0| 66|
|  2|  1| 58|
|  2|  2| 88|
|  2|  3| 72|
|  2|  4|  2|
|  3|  0| 87|
|  3|  1| 54|
|  3|  2| 96|
|  3|  3| 90|
|  3|  4|  6|
+---+---+---+

